<div style="color:#006666; padding:0px 10px; border-radius:5px; font-size:18px;"><h1 style='margin:10px 5px'>Useful Methods</h1>
</div>

© Copyright Machine Learning Plus

<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Random Sampling and Shuffling</h2>
</div>

__Task__: How to pick random sample of observations from the dataframe? 

(Useful for performing cross validation experiments when building ML models)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("Datasets/Property_Crimes.csv")
df.head(20)

Randomly sample 70% of observations from `df` without replacement


`random_state` is for repeatability of the randomness.

In [ ]:
nrows = round(df.shape[0]*.7)
df_sample = df.sample(nrows, replace=False, random_state=101)
df_sample

### Bootstrap Sampling

Bootstrap sampling is when you pick the same number of rows as in the dataset but with replacement. You can use `df.sample()` to easily do this.

Used In:
1. In machine learning algorithms such as Random Forest, Bagging.
2. Computing confidence intervals

In [ ]:
df.sample(frac=1, replace=True, random_state=100)

### Challenge

Calculate the 95% confidence interval of the means for the following:


```python
# Input
import numpy as np
np.random.seed(100)
arr = pd.Series(np.random.normal(10, 3, (100)))
arr.head()
```

__Procedure:__
1. Bootstrap sample a large number of times (10000)
2. Calc mean of each sample.
3. Compute the 2.5%ile and 97.5%ile for the lower and upper bounds of the confidence intervals.


In [ ]:
# Input
import numpy as np
np.random.seed(100)
arr = pd.Series(np.random.normal(10, 3, (100)))
arr.head()

In [ ]:
https://git.io/JsnsG

In [ ]:
# Solution
means = []
for i in range(10000):
    means.append(arr.sample(frac=1, replace=True).mean())

# Sort
means = sorted(means)
means = pd.Series(means)

# lower and upper
means.quantile(q=.025), means.quantile(q=.975)   

<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Dummy Variables</h2>
</div>

Dummy variables, also called One-Hot Encoding is a way of converting a categorical variable into as many binary variables as there are categories.

The reason to do this is to allow ML algorithms to understand and use the data.

![image.png](attachment:image.png)
Source: Stackoverflow

In [ ]:
import pandas as pd
df = pd.read_csv("Datasets/Property_Crimes.csv")
df.head()

In [ ]:
pd.get_dummies(df.Group_Name, prefix='Group')

 <div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3. Categorical Data</h2>
</div>

Pandas introduces a new memory efficient datatype for categorical data. It is used to 

1. __Represent a categorical variable__ present in string format. While a string column's (object dtype) memory allocation is dependent on a constant * length of the data, a categorical datatype's memory requirement depends in the number of categories. 

2. Indicate __intrinsic order__ within the categories (ordered data).

3. __Signal other ML__ libraries, that this columns should be treated as categorical data. 

In [ ]:
import pandas as pd
df = pd.read_csv("Datasets/Property_Crimes.csv")
df.head()

In [ ]:
df.info()

__Task__: Convert the `Group_Name` column to a pandas categorical column

In [ ]:
df['Group_Name_Cat'] = df['Group_Name'].astype('category')
df.head()

In [ ]:
df.info()

__Check the datatype__

In [ ]:
# Categorical
df['Group_Name_Cat'].dtype

In [ ]:
# Object
df['Group_Name'].dtype

Alternately, you can use __`pd.categorical`__ to create the variable from scratch. 

In [ ]:
cat = pd.Categorical(df['Group_Name'], categories=df['Group_Name'].unique())
cat

Internally, the categories are stored as numerical codes, each code points to a specific category.

In [ ]:
cat.codes

See the categories

In [ ]:
cat.categories

__Question__: Will result of `cat.categories` be always same as `cat.unique()`?

Need not be. Because:

1. `cat.unique()` will ensure the items will be listed in the order of appearance.
2. `cat.categories` may contain categories that are not actually part of data. Not the case with `cat.unique()`.

If your data is a series, the categorical attributes are available under `series.cat.xyz`. Ex: `series.cat.categories`


In [ ]:
ser = pd.Series(['a','b','c','d', 'a', 'b'], dtype="category")
ser

In [ ]:
# ser.categories  --> Wont work
ser.cat.categories

__Add a new category__

In [ ]:
ser = ser.cat.add_categories('missing')
ser

__Remove unused categories__

It's possible that some cateogories may not actually be present in data.

In [ ]:
ser.cat.remove_unused_categories()

__Remove even used categories__

In [ ]:
ser.cat.remove_categories('d')

__Ordered Categorical__ variables have an intrinsic ordering of the categories.

In [ ]:
ser = pd.Series(pd.Categorical(["a", "b", "c", "a"], ordered=True))
ser

In [ ]:
ser.cat.codes

In [ ]:
ser = ser.cat.reorder_categories(['c', 'b', 'a'], ordered=True)
ser

__Sorting__

On sorting, the values will be rearranged as per the intrinsic ordering.

In [ ]:
ser.sort_values(inplace=True, ascending=False)
ser

<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4. Method Chaining</h2>
</div>

Method chaining is a programming style where you implement a pipeline of functions in a single call.

__Advantage:__
1. Makes the code easy to read, in the sequence it happens.
2. Makes your codebase compact and organised.

__Disadvantage:__
1. Harder to debug. 

In [ ]:
import pandas as pd
df = pd.read_csv("Datasets/Property_Crimes.csv")

# rename columns
df.columns = df.columns.str.lower()

# recreate Subgroup
df['sub_group_name']  = df['group_name'].str.split("-", expand=True).loc[:, 0]


# Convert Area_Name, Group_Name and Sub_Group_Name to categorical
df['area_name_cat']      = pd.Categorical(df['area_name']) 
df['group_name_cat']     = pd.Categorical(df['group_name'])
df['sub_group_name_cat'] = pd.Categorical(df['sub_group_name'])


# Create Total Cases Lost
df['cases_lost'] = df['cases_property_stolen'] - df['cases_property_recovered']

# Create Total Value of Lost
df['value_lost'] = df['value_of_property_stolen'] - df['value_of_property_recovered']

df.sample(5)

Instead of having one call, you can pipe them all together in one function pipeline.

In [ ]:
def to_categorical(df, column_name):
    df[str(column_name)+'_Cat'] = pd.Categorical(df[column_name]) 
    return df


def read_data(filepath):
    df = (pd.read_csv(filepath)
            .rename(columns=str.lower)
            .pipe(to_categorical, 'area_name')
            .pipe(to_categorical, 'group_name')
            .pipe(to_categorical, 'sub_group_name')
            .assign(cases_lost = lambda x: x['cases_property_stolen'] - x['cases_property_recovered'])
            .assign(value_lost = lambda x: x['value_of_property_stolen'] - x['value_of_property_recovered']))
    return df

df = read_data("Datasets/Property_Crimes.csv")
df.head()

 <div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>5. Efficiently reading data from multiple CSV files</h2>
</div>

__Task:__ 

Sometimes your data is not present in just one file but split in multiple files. And you want to read them all and combine into one single dataframe. 



__Approach 1:__

In [ ]:
import glob
csvfiles = []

# provide path in relation to working directory or fill folder path
csvfiles = glob.glob("Datasets/AReM/lying\*.csv")
print(*csvfiles, sep="\n")


Datasets/AReM/lying\dataset1.csv
Datasets/AReM/lying\dataset10.csv
Datasets/AReM/lying\dataset11.csv
Datasets/AReM/lying\dataset12.csv
Datasets/AReM/lying\dataset13.csv
Datasets/AReM/lying\dataset14.csv
Datasets/AReM/lying\dataset15.csv
Datasets/AReM/lying\dataset2.csv
Datasets/AReM/lying\dataset3.csv
Datasets/AReM/lying\dataset4.csv
Datasets/AReM/lying\dataset5.csv
Datasets/AReM/lying\dataset6.csv
Datasets/AReM/lying\dataset7.csv
Datasets/AReM/lying\dataset8.csv
Datasets/AReM/lying\dataset9.csv


__Read All files and Append to one Data Frame__

In [ ]:
import os
import pandas as pd

list_df = []


for csvfile in csvfiles:
    fpath = csvfile.replace("\\","/")
    print("Reading: ", fpath.ljust(40), "Exists: ", os.path.exists(fpath))
    df = pd.read_csv(fpath, skiprows=4, header=0)

    # Add filename column
    csv_name = csvfile.split('\\')[-1].split('.')[0]
    df['file'] = csv_name
    
    # Add df to a list
    list_df.append(df)
    
#concat all the df in the list
final_df = pd.concat(list_df)

Reading:  Datasets/AReM/lying/dataset1.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset10.csv        Exists:  True
Reading:  Datasets/AReM/lying/dataset11.csv        Exists:  True
Reading:  Datasets/AReM/lying/dataset12.csv        Exists:  True
Reading:  Datasets/AReM/lying/dataset13.csv        Exists:  True
Reading:  Datasets/AReM/lying/dataset14.csv        Exists:  True
Reading:  Datasets/AReM/lying/dataset15.csv        Exists:  True
Reading:  Datasets/AReM/lying/dataset2.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset3.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset4.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset5.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset6.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset7.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset8.csv         Exists:  True
Reading:  Datasets/AReM/lying/dataset9.csv         Exists:  True


In [ ]:
final_df.head()

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,file
0,0,29.0,0.0,9.00,0.71,8.50,0.50,dataset1
1,250,29.0,0.0,8.00,0.71,8.50,0.50,dataset1
2,500,29.0,0.0,8.00,0.71,8.00,1.00,dataset1
3,750,28.5,0.5,8.25,0.43,8.75,0.43,dataset1
4,1000,29.0,0.0,8.75,1.09,9.00,0.00,dataset1


In [ ]:
final_df.shape

(7200, 8)

In [ ]:
df.columns

__Approach 2: Generator Approach__

Put it all in one function call.

In [ ]:
df = pd.concat(pd.read_csv(fpath, skiprows=4, header=0) for fpath in csvfiles)
df

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,29.00,0.00,9.00,0.71,8.50,0.50
1,250,29.00,0.00,8.00,0.71,8.50,0.50
2,500,29.00,0.00,8.00,0.71,8.00,1.00
3,750,28.50,0.50,8.25,0.43,8.75,0.43
4,1000,29.00,0.00,8.75,1.09,9.00,0.00
...,...,...,...,...,...,...,...
475,118750,41.50,0.50,10.67,0.47,14.00,0.82
476,119000,41.50,0.50,10.80,0.40,14.40,0.80
477,119250,41.75,0.43,10.00,0.00,13.67,0.94
478,119500,42.00,0.00,9.40,0.49,14.00,1.10


If you care about adding the filename as a new column, define it in a function function.

In [ ]:
def read(fpath):
    df = pd.read_csv(fpath, skiprows=4, header=0)
    csv_name = csvfile.split('/')[-1].split('.')[0]
    df['file'] = csv_name
    return df

In [ ]:
df = pd.concat(read(fpath) for fpath in csvfiles)
df

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,file
0,0,29.00,0.00,9.00,0.71,8.50,0.50,lying\dataset9
1,250,29.00,0.00,8.00,0.71,8.50,0.50,lying\dataset9
2,500,29.00,0.00,8.00,0.71,8.00,1.00,lying\dataset9
3,750,28.50,0.50,8.25,0.43,8.75,0.43,lying\dataset9
4,1000,29.00,0.00,8.75,1.09,9.00,0.00,lying\dataset9
...,...,...,...,...,...,...,...,...
475,118750,41.50,0.50,10.67,0.47,14.00,0.82,lying\dataset9
476,119000,41.50,0.50,10.80,0.40,14.40,0.80,lying\dataset9
477,119250,41.75,0.43,10.00,0.00,13.67,0.94,lying\dataset9
478,119500,42.00,0.00,9.40,0.49,14.00,1.10,lying\dataset9


### Challenge

__1.__ Read only the following two columns ('avg_rss12', 'var_rss12') and the first 10 observations from each file in `Datasets/AReM/lying` directory and append as one dataframe `df`.

__2.__ Then, re-distribute and store this df into 15 separate csv files.